# Ejercicio Stream Cipher

Diego Andrés Morales Aquino - 21762

In [231]:
def bytes_str(data):
    return ''.join(f'{byte:08b}' for byte in data)
    
def print_bin(data):
    print(bytes_str(data))

## 1. Generación de Keystream

In [232]:
def linear_congruential_gen(seed, a, c, m):
    """
    Generador de números aleatorios: generador congruencial lineal
    https://www.geeksforgeeks.org/pseudo-random-number-generator-prng/
    """

    if not (0 <= seed < m and 0 < a < m and 0 <= c < m and m > 0):
        raise ValueError("Parámetros inválidos: deben cumplir 0 ≤ seed < m, 0 < a < m, 0 ≤ c < m, m > 0")

    
    # Generar números aleatorios: Xn+1 = (a * Xn + c) mod m
    return (a * seed + c) % m


In [233]:
# Ejemplo de generación de números aleatorios

m = 2**31 - 1
a = 1664525
c = 1013904223
seed = 42

for i in range(10):
    seed = linear_congruential_gen(seed, a, c, m)
    print(seed)


1083814273
379334258
637876145
791926961
1941559326
1140840309
717226111
1959858023
2018898833
1629026422


In [234]:
def generate_keystream(seed, n_bytes):
    """
    Generar secuencia de bytes pseudoaleatorios
    """
    rand = seed
    keystream = []

    # Establecere parametros a, c y m por defecto
    m = 2**31 - 1
    a = 1664525
    c = 1013904223
    
    for _ in range(n_bytes):
        rand = linear_congruential_gen(rand, a, c, m)
        keystream.append(rand % 256)
    
    return bytes(keystream)

In [235]:
# Ejemplo de generación de keystreams pseudoaleatorios

seed = 42

keystream = generate_keystream(seed, 10)
print_bin(keystream) # Mostrar keystream en binario

10000001011100101011000110110001000111100111010101111111011001111001000101110110


## 2. Cifrado

In [236]:
def cypher(original_text, seed):
    """
    Cifrado: texto XOR keystream
    """
    data_bytes = original_text.encode('utf-8')
    
    keystream = generate_keystream(seed, len(data_bytes))
    return bytes([d ^ k for d, k in zip(data_bytes, keystream)])

In [237]:
# Ejemplo de cifrado
seed = 42
text = "Diego Morales, 21762"
text_cypher_bin = cypher(text, seed)
print("seed: ", seed)
print("Texto original: ", text)
print("Texto cifrado: ")
print("(Bytes): ", text_cypher_bin)
print("(Binario): ", bytes_str(text_cypher_bin))

seed:  42
Texto original:  Diego Morales, 21762
Texto cifrado: 
(Bytes):  b'\xc5\x1b\xd4\xd6qU2\x08\xe3\x17\xcbX\x13P*/ Ol\x0c'
(Binario):  1100010100011011110101001101011001110001010101010011001000001000111000110001011111001011010110000001001101010000001010100010111100100000010011110110110000001100


## 3. Descifrado

In [238]:
def decypher(data_bytes, seed):
    """
    Descifrado: texto_cifrado XOR keystream
    """
    if not isinstance(data_bytes, bytes):
        raise ValueError("data debe ser de tipo bytes y no de tipo {}".format(type(data_bytes)))
    
    keystream = generate_keystream(seed, len(data_bytes))
    decypher_bytes = bytes([d ^ k for d, k in zip(data_bytes, keystream)])

    # Remover el padding (bytes nulos al final) solo si fue agregado
    if len(decypher_bytes) % 8 == 0:
        decypher_bytes = decypher_bytes.rstrip(b'\\x00')
    
    return decypher_bytes.decode('utf-8')


In [239]:
# Ejemplo de descifrado

seed = 42

text_decypher = decypher(text_cypher_bin, seed)
print("seed: ", seed)
print("Texto descifrado: ", text_decypher)

seed:  42
Texto descifrado:  Diego Morales, 21762


## Preguntas

**¿Qué sucede cuando cambias la clave utilizada para generar el keystream?**

Cuando se cambia la clave (semilla) utilizada para generar el keystream, se está alterando la secuencia de números pseudoaleatorios que se utilizó para construir cada uno de los bytes de un keystream dado. Por ejemplo, para una semillas X, se generaron los número pseudoaleatorios X+1, x+2, ... x+n. Dichos números fueron utilizados para construir el keystream de n bytes. Sin embargo, si la clave (semilla) inicial es modificada por un número Y diferente de X, la secuencia de números Y+1, Y+2, ..., Y+n será diferente a la secuencia anterior (al menos en la mayoría de ocasiones), por lo que se obtendrá un keystream distinto. 

En consecuencia, al obtener un keystream distinto, al aplicar la operación XOR con el mensaje cifrado, no será posible obtener el mensaje original(en la mayoria de los casos).

**¿Qué riesgos de seguridad existen si reutilizas el mismo keystream para cifrar dos mensajes diferentes?**

Cuando se tiene dos mensajes cifrados con el mismo keystream, existe una gran vulnerabilidad que permite descifrar el mensaje sin la necesidad de conocer el keystream con el cual fueron cifrados. Esto se debe a la propiedad de XOR:

msg_cifrado_1 XOR msg_cifrado_1 = (msg_1 XOR keystream) XOR (msg_2 XOR keystream)

Lo cual equivale a:

msg_cifrado_1 XOR msg_cifrado_2 = msg_1 XOR msg_2

Esto quiere decir que, si se tienen dos mensajes cifrados con el mismo keystream y se les aplica una operación XOR, el resultado será equivalente a un XOR entre los mensajes originales. Esto significa que, si se conoce uno de los mensajes originales, al aplicar XOR con el resultado obtenido, es posible descifrar el otro mensaje sin necesidad de conocer el keystream.


Fuente: http://www.crypto-it.net/eng/attacks/two-time-pad.html



**¿Cómo afecta la longitud del keystream a la seguridad del cifrado?**

Mientras mayor sea la longitud del keystream, se requerirá una mayor cantidad de números aleatorios para su generación, lo que reduce la probabilidad de patrones repetitivos o ciclos predecibles en la secuencia. Además, un keystream lo suficientemente largo, posee una mayor cantidad de combinaciones posibles, lo cual dificulta su predicción por parte de un atacante y fortalece la seguridad del cifrado.

Por otro lado, si el keystream es demasiado corto, corre el riesgo de repetirse al cifrar múltiples mensajes y poder descifrar mensajes como se mostró anteriormente. Así como, un keystream corto es más fácil de predecir, pues se reduce la cantidad de posibles combinaciones.

**¿Qué consideraciones debes tener al generar un keystream en un entorno real?**

Para aumentar la seguridad y confiabilidad del cifrado en un entorno real, es necesario garantizar un alto grado de aleatoriedad y entropía en el keystream utilizado. Esto para evitar patrones predecibles o keystreams fáciles de encontrar. 

Por otro lado, es importante que la longitud de bits del keystream coincida con el mensaje que se está cifrando, o que este sea lo suficientemente largo para que no se tenga que repetir. Si se opta por repetir el keystream, se podría abrir una vulnerabilidad como la mencionada anteriormente, con 2 mensajes parciales cifrados con una misma key.

Además, es importante que la clave utilizada para generar el keystream (en este caso la semilla) sea única y esté protegida para evitar su exposición. Debe ser almacenada de manera segura y transmitida solo a través de canales cifrados.

## Ejemplos y pruebas unitarias

In [240]:
seed = 53
text = "Pachón morado con agua."

text_cypher_bin = cypher(text, seed)
print("Seed:", seed, "\nTexto:", text)
print("Texto cifrado:")
print("(Bytes) ",text_cypher_bin)
print("(Bits) ", bytes_str(text_cypher_bin))

if text_cypher_bin != b'@D\x17\xb3\xf2\xcaO\x03\xf2\x07h\xbb\xc7\x8b\x05\xe01\x8f\x97\xe8Y~r\x91':
    raise ValueError("Error en el cifrado")

print("Test de cifrado exitoso")

text_decypher = decypher(text_cypher_bin, seed)
print("\nTexto descifrado: ", text_decypher)
if text_decypher != text:
    raise ValueError("El texto descifrado no coincide con el texto original")

print("Test de descifrado exitoso")


Seed: 53 
Texto: Pachón morado con agua.
Texto cifrado:
(Bytes)  b'@D\x17\xb3\xf2\xcaO\x03\xf2\x07h\xbb\xc7\x8b\x05\xe01\x8f\x97\xe8Y~r\x91'
(Bits)  010000000100010000010111101100111111001011001010010011110000001111110010000001110110100010111011110001111000101100000101111000000011000110001111100101111110100001011001011111100111001010010001
Test de cifrado exitoso

Texto descifrado:  Pachón morado con agua.
Test de descifrado exitoso


In [241]:
seed = 6598
text = "Atacar al amanecer. ¡Viva la revolución!"
text_cypher_bin = cypher(text, seed)
print("Seed:", seed, "\nTexto:", text)
print("Texto cifrado:")
print("(Bytes) ",text_cypher_bin)
print("(Bits) ", bytes_str(text_cypher_bin))

if text_cypher_bin != b'3_\xb2\xd6\x8a\xb3\xcb\x04\xa7\x90>\xacrtuC\x84\x1d;\x92\xe1(\xf7oB\x01\x8a\xb7s\xba\xcf\x1b+\xe5\xd0\x91\xa9#\xfa<tP':
    raise ValueError("Error en el cifrado")

print("Test de cifrado exitoso")

text_decypher = decypher(text_cypher_bin, seed)
print("\nTexto descifrado: ", text_decypher)
if text_decypher != text:
    raise ValueError("El texto descifrado no coincide con el texto original")

print("Test de descifrado exitoso")

Seed: 6598 
Texto: Atacar al amanecer. ¡Viva la revolución!
Texto cifrado:
(Bytes)  b'3_\xb2\xd6\x8a\xb3\xcb\x04\xa7\x90>\xacrtuC\x84\x1d;\x92\xe1(\xf7oB\x01\x8a\xb7s\xba\xcf\x1b+\xe5\xd0\x91\xa9#\xfa<tP'
(Bits)  001100110101111110110010110101101000101010110011110010110000010010100111100100000011111010101100011100100111010001110101010000111000010000011101001110111001001011100001001010001111011101101111010000100000000110001010101101110111001110111010110011110001101100101011111001011101000010010001101010010010001111111010001111000111010001010000
Test de cifrado exitoso

Texto descifrado:  Atacar al amanecer. ¡Viva la revolución!
Test de descifrado exitoso


### Descifrado

In [242]:
text_cypher_bin = b'@D\x17\xb3\xf2\xcaO\x03\xf2\x07h\xbb\xc7\x8b\x05\xe01\x8f\x97\xe8Y~r\x91'
seed = 53

text_decypher = decypher(text_cypher_bin, seed)
print("Seed:", seed)
print("Texto descifrado:", text_decypher)

if text_decypher != "Pachón morado con agua.":
    raise ValueError("Error en el descifrado")

print("Test de descifrado indirecto exitoso")

Seed: 53
Texto descifrado: Pachón morado con agua.
Test de descifrado indirecto exitoso


In [243]:
text_cypher_bin = b'3_\xb2\xd6\x8a\xb3\xcb\x04\xa7\x90>\xacrtuC\x84\x1d;\x92\xe1(\xf7oB\x01\x8a\xb7s\xba\xcf\x1b+\xe5\xd0\x91\xa9#\xfa<tP'
seed = 6598

text_decypher = decypher(text_cypher_bin, seed)
print("Seed:", seed)
print("Texto descifrado:", text_decypher)

if text_decypher != "Atacar al amanecer. ¡Viva la revolución!":
    raise ValueError("Error en el descifrado")

print("Test de descifrado indirecto exitoso")

Seed: 6598
Texto descifrado: Atacar al amanecer. ¡Viva la revolución!
Test de descifrado indirecto exitoso


## Reflexiona sobre las limitaciones de los generadores pseudoaleatorios simples en la seguridad de cifrados reales.

Tal y como se realizó en el ejercicio, el sistema de cifrado está basado principalmente en la aleatoriedad de las keystreams. Sin embargo, dado que estos algoritmos de generación de números pseudoaleatorios son deterministas, basta con conocer la semilla del generador para poder producir toda la secuencia de números y hallar la clave en cuestión. 

Por otro lado, si el algoritmo no es lo suficientemente robusto, puede tener ciclos predecibles o una entropía limitada. Por ejemplo, en el algoritmo implementado de congruencia lineal, dependiendo de los parámetros elegidos, el algoritmo sigue una secuencia ascendente hasta llegar al límite, así como, si el rango no era lo suficientemente amplio, podía generarse siempre la misma secuencia de números en un ciclo, lo cual resultaría en una cadena con un bajo nivel de aleatoriedad y por lo tanto, más vulnerable a ataques. De la misma forma, al depender de parámetros fijos o simples transformaciones matemáticas, generan secuencias con poca variabilidad. Esto puede hacer que la secuencia generada sea relativamente fácil de adivinar o correlacionar 

De manera que, es de gran importancia utilizar otro tipo de algoritmos que proporcionen una mejor resistencia contra la predicción, fuerza bruta, ciclos y ataques de análisis estadístico, asegurando una mayor complejidad, aleatoriedad e impredecibilidad en la generación de claves.